In [12]:
import numpy as np

%load_ext autoreload
%autoreload 2
import my_baselines
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
with open('train.json') as f:
    data = f.readlines()

import ast
data = [ast.literal_eval(x) for x in data]

In [11]:
data[1]

{'categories': [['Clothing, Shoes & Jewelry',
   'Women',
   'Clothing',
   'Lingerie, Sleep & Lounge',
   'Intimates',
   'Bras',
   'Everyday Bras'],
  ['Clothing, Shoes & Jewelry',
   'Women',
   'Petite',
   'Intimates',
   'Bras',
   'Everyday Bras']],
 'categoryID': 0,
 'helpful': {'nHelpful': 0, 'outOf': 0},
 'itemID': 'I697650540',
 'rating': 4.0,
 'reviewHash': 'R436443063',
 'reviewText': 'I love the look of this bra, it is what I wanted, however, it is about a cup size AND band size too small. The cups are sheer, which is what I wanted and the look is very sexy and it arrived much quicker than promised. I plan to order another one, but in a larger size.',
 'reviewTime': '02 7, 2013',
 'reviewerID': 'U714157797',
 'summary': 'Beautiful but size runs small',
 'unixReviewTime': 1360195200}

In [3]:
from io import open
import unicodedata
import string
import random
import re

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [10]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


MAX_LENGTH = 15

def filterSummaries(summaries):
    return [s for s in summaries if len(s)<MAX_LENGTH]

In [8]:
summaries = [[normalizeString(s) for s in d['summary'].split()] for d in data]
lang = Lang('Summary')
len(summaries)

In [9]:
summaries = filterSummaries(summaries)
len(summaries)

198939

In [13]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


# def tensorsFromPair(pair):
#     input_tensor = tensorFromSentence(input_lang, pair[0])
#     target_tensor = tensorFromSentence(output_lang, pair[1])
#     return (input_tensor, target_tensor)

In [14]:
# criterion = nn.MSELoss()

def train(input_tensor, target_tensor, model, optimizer, criterion):
    
    hidden = model.initHidden()
    optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    outputs = torch.zeros(model.hidden_size, device=device)

    loss = 0
    for ei in range(input_length):
        outputs, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
    
    loss = criterion(outputs, target_tensor)

    loss.backward()
    optimizer.step()
    
    return loss.item()